In [130]:
import pandas as pd
import numpy as np
import quandl
import os
import warnings

warnings.filterwarnings('ignore')
quandl.ApiConfig.api_key = os.getenv('QUANDL_APY_KEY')

## Getting necessary data for a given company, today
First step is to get necessary data to determine if a given company pass or fails Ben Graham's test criteria for the *defensive investor*, as described in *Chapter 14 of Intelligent Investor*. Using [Quandl SF1 | Core US Fundamentals Data](https://www.quandl.com/databases/SF1)

In [166]:
ticker = 'MSFT'

def get_data(ticker): 
    columns = ['calendardate', 'datekey', 'revenue', 'currentratio', 'eps', 'dps', 'bvps', 'price']
    df = quandl.get_table('SHARADAR/SF1', dimension='MRY', ticker='NWL', qopts={"columns":columns})
    df.sort_values('calendardate', inplace=True)
    return df

get_data(ticker)

,calendardate,datekey,revenue,currentratio,eps,dps,bvps,price
None,,,,,,,,
22,1995-12-31,1995-12-31,2498414000,1.665,1.41,0.460,8.191,25.88
21,1996-12-31,1996-12-31,2972839000,1.725,1.60,0.560,9.268,31.50
20,1997-12-31,1997-12-31,5641441000,1.810,1.15,0.640,16.401,42.50
19,1998-12-31,1998-12-31,6493172000,2.091,1.72,0.720,10.116,41.25
18,1999-12-31,1999-12-31,6711768000,1.680,0.34,0.800,9.564,29.00
17,2000-12-31,2000-12-31,6934700000,1.857,1.57,0.840,9.190,22.75
16,2001-12-31,2001-12-31,6909300000,1.125,0.99,0.840,9.114,27.57
15,2002-12-31,2002-12-31,6436900000,1.178,-0.76,0.840,7.717,30.33
14,2003-12-31,2003-12-31,6614800000,1.484,-0.17,0.840,7.348,22.77


## Check if a given company pass the 7 criteria
Second step is to implement the criteria and check if a given company pass it or not.

In [163]:
import humanize

def criteria_defensive_investor(data, verbose=False):
    # In case of no data
    if data.shape[0] == 0:  # No data
        if verbose:
            print('- No data available')
        return False
    
    # Size criteria
    size_criteria = data['revenue'].values[-1] > 100000000
    
    # Financial condition criteria
    if data['currentratio'].values[-1] is not None:
        current_ratio = data['currentratio'].values[-1]
        financial_condition_criteria = current_ratio > 2
    else:
        current_ratio = float('NaN')
        financial_condition_criteria = False
    
    # Earnings stability criteria
    earnings_stability_criteria = (data['eps'].tail(10) > 0).all()
    
    # Dividends record criteria
    dividend_record_criteria = (data['dps'].tail(20) > 0).all()
    
    # Earnings growth criteria
    last_year = pd.to_datetime(data['calendardate'].values[-1]).year
    eps_0 = data[(data['calendardate'].dt.year > last_year - 13) & 
                 (data['calendardate'].dt.year <= last_year - 10)]['eps'].mean()
    eps_1 = data[(data['calendardate'].dt.year > last_year - 3) & 
                 (data['calendardate'].dt.year <= last_year)]['eps'].mean()
    earnings_growth_criteria = (np.float64(eps_1) / eps_0) > 1.33
    
    # P/E ratio criteria
    current_price = data['price'].values[-1]
    pe = current_price / eps_1
    pe_criteria = pe < 15
    
    # Price to Assets criteria
    pb = current_price / data['bvps'].values[-1]
    pb_criteria = pb < 1.5
    if (pe * pb < 22.5):
        pb_criteria = True
    
    full_criteria = size_criteria and financial_condition_criteria and earnings_stability_criteria \
        and dividend_record_criteria and earnings_growth_criteria and pe_criteria
    
    if verbose:
        print('- Size criteria: \t\t' + str(size_criteria) + 
              '\tRevenues of $' + humanize.intword(data['revenue'].values[-1]) + ' (threshold is $100 million)')
        print('- Financial condition criteria: ' + str(financial_condition_criteria) + 
              '\tCurrent ratio of %1.2f' % current_ratio + ' (threshold is 2.0)')
        print('- Earnings stability criteria: \t' + str(earnings_stability_criteria) + 
              '\tPositive earnings in %d of past 10 years' % data.tail(10)[data['eps'] > 0]['eps'].count())
        print('- Dividend record criteria: \t' + str(dividend_record_criteria) + 
              '\tDistribution of dividend in %d of past 20 years' % data.tail(20)[data['dps'] > 0]['dps'].count())
        print('- Earnings growth criteria: \t' + str(earnings_growth_criteria) + 
              '\tEarnings change of %+.0f%%' % (100 * ((np.float64(eps_1) / eps_0) - 1)) + ' in past 10 years (minimum is +33%)')
        print('- Moderate P/E ratio criteria: \t' + str(pe_criteria) + 
              '\tCurrent price is %1.1fx avg P3yrs earnings (limit is 15)' % pe)
        print('- Moderate P/B ratio criteria: \t' + str(pb_criteria) + 
              '\tCurrent price is %1.1fx last book value (limit 1.5), \n\t\t\t\t\tand PE * PB is %1.1f (limit 22.5)' % (pb, pe * pb))
        print('- Full criteria: \t\t' + str(full_criteria))
    
    return full_criteria

In [160]:
%%capture cap --no-stderr

UsageError: %%capture is a cell magic, but the cell body is empty.


In [115]:
criteria_defensive_investor(df, verbose=True)

with open('output.txt', 'w') as f:
    f.write(cap.stdout)

- Size criteria: 		True	Reveneus of $110.4 billion (threshold is $100 million)
- Financial condition criteria: True	Current ratio of 2.90 (threshold is 2.0)
- Earnings stability criteria: 	True	Positive earnings in 10 of past 10 years
- Dividend record criteria: 	False	Distribution of dividend in 16 of past 20 years
- Earnings growth criteria: 	True	Earnings change of +76% in past 10 years (minimum is +33%)
- Moderate P/E ratio criteria: 	False	Current price is 36.8x avg P3yrs earnings (limit is 15)
- Moderate P/B ratio criteria: 	False	Current price is 9.2x last book value (limit 1.5), 
					and PE * PB is 337.4 (limit 22.5)
- Full criteria: 			False


## Bulk test it in DJIA companies

In [125]:
def get_djia_symbols():
    data = pd.read_html("https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average")
    table = data[1]
    sliced_table = table[1:]
    header = table.iloc[0]
    corrected_table = sliced_table.rename(columns=header)
    return corrected_table['Symbol'].tolist()

def get_sp500_symbols():
    data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    table = data[1]
    sliced_table = table[1:]
    header = table.iloc[0]
    corrected_table = sliced_table.rename(columns=header)
    return corrected_table['Symbol'].tolist()

In [126]:
%%capture cap --no-stderr

djia_list = get_djia_symbols()
columns = ['ticker', 'calendardate', 'datekey', 'revenue', 'currentratio', 'eps', 'dps', 'bvps', 'price']
df_all = quandl.get_table('SHARADAR/SF1', dimension='MRY', ticker=djia_list, qopts={"columns":columns})

for ticker in djia_list:
    print('\nTest for ' + ticker + ':')
    df = df_all[df_all['ticker'] == ticker]
    df.sort_values('calendardate', inplace=True)
    criteria_defensive_investor(df, verbose=True)
    
with open('output.txt', 'w') as f:
    f.write(cap.stdout)

## Bulk test it in S&P500 companies

In [164]:
#%%capture cap --no-stderr

spy_list = get_sp500_symbols()
columns = ['calendardate', 'datekey', 'revenue', 'currentratio', 'eps', 'dps', 'bvps', 'price']

positive_companies = []
for ticker in spy_list:
    print('\nTest for ' + ticker + ':')
    df = quandl.get_table('SHARADAR/SF1', dimension='MRY', ticker=ticker, qopts={"columns":columns})
    df.sort_values('calendardate', inplace=True)
    if criteria_defensive_investor(df, verbose=True):
        positive_companies.append(ticker)
    
print('\nCompanies meeting criteria: ' + str(positive_companies))

#with open('output.txt', 'w') as f:
#    f.write(cap.stdout)


Test for MMM:
- Size criteria: 		True	Revenues of $31.7 billion (threshold is $100 million)
- Financial condition criteria: False	Current ratio of 1.86 (threshold is 2.0)
- Earnings stability criteria: 	True	Positive earnings in 10 of past 10 years
- Dividend record criteria: 	True	Distribution of dividend in 20 of past 20 years
- Earnings growth criteria: 	True	Earnings change of +62% in past 10 years (minimum is +33%)
- Moderate P/E ratio criteria: 	False	Current price is 29.2x avg P3yrs earnings (limit is 15)
- Moderate P/B ratio criteria: 	False	Current price is 12.1x last book value (limit 1.5), 
					and PE * PB is 354.2 (limit 22.5)
- Full criteria: 		False

Test for ABT:
- Size criteria: 		True	Revenues of $27.4 billion (threshold is $100 million)
- Financial condition criteria: True	Current ratio of 2.26 (threshold is 2.0)
- Earnings stability criteria: 	True	Positive earnings in 10 of past 10 years
- Dividend record criteria: 	True	Distribution of dividend in 20 of past 20 y

- Size criteria: 		True	Revenues of $8.9 billion (threshold is $100 million)
- Financial condition criteria: True	Current ratio of 2.17 (threshold is 2.0)
- Earnings stability criteria: 	True	Positive earnings in 10 of past 10 years
- Dividend record criteria: 	True	Distribution of dividend in 20 of past 20 years
- Earnings growth criteria: 	True	Earnings change of +91% in past 10 years (minimum is +33%)
- Moderate P/E ratio criteria: 	False	Current price is 21.3x avg P3yrs earnings (limit is 15)
- Moderate P/B ratio criteria: 	False	Current price is 3.4x last book value (limit 1.5), 
					and PE * PB is 71.9 (limit 22.5)
- Full criteria: 		False

Test for AKAM:
- Size criteria: 		True	Revenues of $2.5 billion (threshold is $100 million)
- Financial condition criteria: True	Current ratio of 2.95 (threshold is 2.0)
- Earnings stability criteria: 	True	Positive earnings in 10 of past 10 years
- Dividend record criteria: 	False	Distribution of dividend in 0 of past 20 years
- Earnings gro

- No data available

Test for MO:
- Size criteria: 		True	Revenues of $25.6 billion (threshold is $100 million)
- Financial condition criteria: False	Current ratio of 0.64 (threshold is 2.0)
- Earnings stability criteria: 	True	Positive earnings in 10 of past 10 years
- Dividend record criteria: 	True	Distribution of dividend in 20 of past 20 years
- Earnings growth criteria: 	False	Earnings change of -1% in past 10 years (minimum is +33%)
- Moderate P/E ratio criteria: 	True	Current price is 14.0x avg P3yrs earnings (limit is 15)
- Moderate P/B ratio criteria: 	False	Current price is 8.9x last book value (limit 1.5), 
					and PE * PB is 124.4 (limit 22.5)
- Full criteria: 		False

Test for AMZN:
- Size criteria: 		True	Revenues of $177.9 billion (threshold is $100 million)
- Financial condition criteria: False	Current ratio of 1.04 (threshold is 2.0)
- Earnings stability criteria: 	False	Positive earnings in 8 of past 10 years
- Dividend record criteria: 	False	Distribution of divide

- Size criteria: 		True	Revenues of $7.0 billion (threshold is $100 million)
- Financial condition criteria: True	Current ratio of 2.95 (threshold is 2.0)
- Earnings stability criteria: 	True	Positive earnings in 10 of past 10 years
- Dividend record criteria: 	False	Distribution of dividend in 13 of past 20 years
- Earnings growth criteria: 	True	Earnings change of +217% in past 10 years (minimum is +33%)
- Moderate P/E ratio criteria: 	False	Current price is 36.2x avg P3yrs earnings (limit is 15)
- Moderate P/B ratio criteria: 	False	Current price is 6.7x last book value (limit 1.5), 
					and PE * PB is 243.5 (limit 22.5)
- Full criteria: 		False

Test for APC:
- Size criteria: 		True	Revenues of $11.9 billion (threshold is $100 million)
- Financial condition criteria: False	Current ratio of 1.73 (threshold is 2.0)
- Earnings stability criteria: 	False	Positive earnings in 4 of past 10 years
- Dividend record criteria: 	True	Distribution of dividend in 20 of past 20 years
- Earnings

- Size criteria: 		True	Revenues of $13.0 billion (threshold is $100 million)
- Financial condition criteria: True	Current ratio of 2.26 (threshold is 2.0)
- Earnings stability criteria: 	False	Positive earnings in 4 of past 10 years
- Dividend record criteria: 	True	Distribution of dividend in 20 of past 20 years
- Earnings growth criteria: 	False	Earnings change of -117% in past 10 years (minimum is +33%)
- Moderate P/E ratio criteria: 	True	Current price is -23.2x avg P3yrs earnings (limit is 15)
- Moderate P/B ratio criteria: 	True	Current price is 2.7x last book value (limit 1.5), 
					and PE * PB is -62.0 (limit 22.5)
- Full criteria: 		False

Test for ANET:
- Size criteria: 		True	Revenues of $1.6 billion (threshold is $100 million)
- Financial condition criteria: True	Current ratio of 4.28 (threshold is 2.0)
- Earnings stability criteria: 	True	Positive earnings in 7 of past 10 years
- Dividend record criteria: 	False	Distribution of dividend in 0 of past 20 years
- Earnings g

...

- Size criteria: 		True	Revenues of $28.9 billion (threshold is $100 million)
- Financial condition criteria: False	Current ratio of nan (threshold is 2.0)
- Earnings stability criteria: 	True	Positive earnings in 10 of past 10 years
- Dividend record criteria: 	True	Distribution of dividend in 20 of past 20 years
- Earnings growth criteria: 	True	Earnings change of +85% in past 10 years (minimum is +33%)
- Moderate P/E ratio criteria: 	True	Current price is 14.1x avg P3yrs earnings (limit is 15)
- Moderate P/B ratio criteria: 	True	Current price is 1.6x last book value (limit 1.5), 
					and PE * PB is 22.0 (limit 22.5)
- Full criteria: 		False

Test for TRIP:
- Size criteria: 		True	Revenues of $1.6 billion (threshold is $100 million)
- Financial condition criteria: True	Current ratio of 2.67 (threshold is 2.0)
- Earnings stability criteria: 	False	Positive earnings in 8 of past 10 years
- Dividend record criteria: 	False	Distribution of dividend in 0 of past 20 years
- Earnings grow

- Size criteria: 		True	Revenues of $59.8 billion (threshold is $100 million)
- Financial condition criteria: False	Current ratio of 1.35 (threshold is 2.0)
- Earnings stability criteria: 	True	Positive earnings in 10 of past 10 years
- Dividend record criteria: 	True	Distribution of dividend in 20 of past 20 years
- Earnings growth criteria: 	True	Earnings change of +83% in past 10 years (minimum is +33%)
- Moderate P/E ratio criteria: 	False	Current price is 18.5x avg P3yrs earnings (limit is 15)
- Moderate P/B ratio criteria: 	False	Current price is 3.4x last book value (limit 1.5), 
					and PE * PB is 63.0 (limit 22.5)
- Full criteria: 		False

Test for UHS:
- Size criteria: 		True	Revenues of $10.4 billion (threshold is $100 million)
- Financial condition criteria: False	Current ratio of 0.97 (threshold is 2.0)
- Earnings stability criteria: 	True	Positive earnings in 10 of past 10 years
- Dividend record criteria: 	False	Distribution of dividend in 15 of past 20 years
- Earnings

- Size criteria: 		True	Revenues of $2.1 billion (threshold is $100 million)
- Financial condition criteria: False	Current ratio of nan (threshold is 2.0)
- Earnings stability criteria: 	True	Positive earnings in 10 of past 10 years
- Dividend record criteria: 	True	Distribution of dividend in 20 of past 20 years
- Earnings growth criteria: 	False	Earnings change of -15% in past 10 years (minimum is +33%)
- Moderate P/E ratio criteria: 	False	Current price is 26.6x avg P3yrs earnings (limit is 15)
- Moderate P/B ratio criteria: 	False	Current price is 3.4x last book value (limit 1.5), 
					and PE * PB is 91.0 (limit 22.5)
- Full criteria: 		False

Test for VMC:
- Size criteria: 		True	Revenues of $3.9 billion (threshold is $100 million)
- Financial condition criteria: True	Current ratio of 2.67 (threshold is 2.0)
- Earnings stability criteria: 	False	Positive earnings in 7 of past 10 years
- Dividend record criteria: 	True	Distribution of dividend in 20 of past 20 years
- Earnings gro

- Size criteria: 		True	Revenues of $16.3 billion (threshold is $100 million)
- Financial condition criteria: False	Current ratio of 1.44 (threshold is 2.0)
- Earnings stability criteria: 	False	Positive earnings in 5 of past 10 years
- Dividend record criteria: 	False	Distribution of dividend in 4 of past 20 years
- Earnings growth criteria: 	False	Earnings change of +nan% in past 10 years (minimum is +33%)
- Moderate P/E ratio criteria: 	False	Current price is 18.4x avg P3yrs earnings (limit is 15)
- Moderate P/B ratio criteria: 	True	Current price is 1.2x last book value (limit 1.5), 
					and PE * PB is 21.9 (limit 22.5)
- Full criteria: 		False

Test for WY:
- Size criteria: 		True	Revenues of $7.2 billion (threshold is $100 million)
- Financial condition criteria: False	Current ratio of 1.47 (threshold is 2.0)
- Earnings stability criteria: 	False	Positive earnings in 8 of past 10 years
- Dividend record criteria: 	True	Distribution of dividend in 19 of past 20 years
- Earnings g

- Size criteria: 		True	Revenues of $5.3 billion (threshold is $100 million)
- Financial condition criteria: True	Current ratio of 3.85 (threshold is 2.0)
- Earnings stability criteria: 	False	Positive earnings in 8 of past 10 years
- Dividend record criteria: 	False	Distribution of dividend in 5 of past 20 years
- Earnings growth criteria: 	False	Earnings change of +nan% in past 10 years (minimum is +33%)
- Moderate P/E ratio criteria: 	False	Current price is 52.7x avg P3yrs earnings (limit is 15)
- Moderate P/B ratio criteria: 	False	Current price is 19.8x last book value (limit 1.5), 
					and PE * PB is 1045.5 (limit 22.5)
- Full criteria: 		False


In [167]:
%%capture cap --no-stderr

spy_list = get_sp500_symbols()
columns = ['calendardate', 'datekey', 'revenue', 'currentratio', 'eps', 'dps', 'bvps', 'price']

positive_companies = []
for ticker in spy_list:
    print('\nTest for ' + ticker + ':')
    df = quandl.get_table('SHARADAR/SF1', dimension='MRY', ticker=ticker, qopts={"columns":columns})
    df.sort_values('calendardate', inplace=True)
    if criteria_defensive_investor(df, verbose=True):
        positive_companies.append(ticker)
    
print('\nCompanies meeting criteria: ' + str(positive_companies))

with open('output.txt', 'w') as f:
    f.write(cap.stdout)

## Next steps:
1. Ajustar a função `criteria_defensive_investor` para receber um ticker ou uma lista de tickers
2. Fazer com que ela devolta um DataFrame com todos os dados, ao invés de imprimir na tela (para um ou múltiplos tickers
3. Parametrizar pela data de hoje variável